In [ ]:
import numpy as np
from hera_cal.io import HERAData
import matplotlib.pyplot as plt
%matplotlib notebook

# HERA Data Analysis Part I: Reading, Manipulating, and Visualizing Visibiltiy Data

### *Workshop Leaders: Josh Dillon & Adam Beardsley*
#### (Originally developed by Carina Cheng & Josh Dillon)

## Reading Files and Metadata

Load HERA data file `zen.2458098.40887.HH.calibrated_downselected.uvh5` (which is in the same folder as this notebook) the `HERAData` object in the `hera_cal.io` module. Then answer these basic questions about the data:

1. How many antennas are in the array and what are their antenna numbers? 


2. How many frequency channels are there in the array. What range do they cover in MHz?


3. What is the width of each frequency channel in MHz?


4. How many time integrations are there in the file?


5. What range of Julian Dates do they span?


6. What is the lengh of each time integration in seconds?


7. What range of LSTs (local sidreal times) do the data span?


8. How many baselines are in the file? How many should there be? Why don't they agree? Hint: check the `history` attribute of the `HERAData` object.

In [ ]:
import numpy as np
from hera_cal.io import HERAData
path = 'zen.2458098.40887.HH.calibrated_downselected.uvh5'
hd = HERAData(path)

In [ ]:
# your answers here (hint: look at the attributes of the HERAData object)
hd.ants

Now let's plot our array to look at where the antennas are. (Hint, use the `hd.antpos` dictionary, `plt.scatter`, to plot antenna positions, and `plt.text` to label them.

In [ ]:
# # Your answer here:
# hd.antpos[136]
# for ant in hd.antpos.keys():
#     print(ant)

# Reading Data and Flags

Using a HERAData object, it's very easy to load in data into a convienent format.

In [ ]:
data, flags, nsamples = hd.read()

In [ ]:
print(type(data))
print(data.keys())

`data`, `flags`, and `nsamples` are all fancy dictionaries (`DataContainer`s) that give us easy access to data. 

In [ ]:
bl = (65, 71, 'xx') # tuple containing baseline and polarization
print(type(data[bl]), data[bl].shape, data[bl].dtype)
print(type(flags[bl]), flags[bl].shape, flags[bl].dtype)

Now we can start plotting unflagged data. If we divide our data by `~flags`, we treat flags as 1s, so we're dividing flagged data by 0. This is usually bad (don't edit your data with your flags!), but it's useful for visualization.

In [ ]:
integration = 30 # this is the integration index, 30 out of 60 total integrations
plt.figure()
plt.plot(hd.freqs/1e6, data[bl][integration,:].real / ~flags[bl][integration,:], 'b-', label='Real part')
plt.plot(hd.freqs/1e6, data[bl][integration,:].imag / ~flags[bl][integration,:], 'r-', label='Imag part')
plt.xlabel('Frequency (MHz)')
plt.ylabel('Visibility (Jy)')
plt.grid(); plt.legend();

Exercise: Plot unflagged visibilities as a function of LST for a few different frequency channels. Label each channel's frequency and include a legend.

In [ ]:
bl = (65, 71, 'xx')
chans = [100,300,750]

# your answer here 

# Waterfall Plots

Another useful way of visualizing our data is to plot it as a waterfall plot. A waterfall plot is a two-dimensional plot of the visibility (the cross-correlated signal between a pair of antennas) as a function of time (y-axis) and frequency (x-axis). We can use the `plt.imshow` to plot the amplitude and phase of the same baseline as above. Note that the keyword `extent` takes in 4 arguments which define the plot axes extent in the order of (xmin, xmax, ymin, ymax), and we've massaged our axes to display frequencies in MHz and times in LST hours.

In [ ]:
import matplotlib 
bl = (65, 71, 'xx')

# Plot absolute value of visibility
plt.figure(figsize=(8,4))
plt.subplot(121)
plt.imshow(np.abs(data[bl]), aspect='auto', norm=matplotlib.colors.LogNorm(1e0,1e2),
          extent=(hd.freqs[0]/1e6, hd.freqs[-1]/1e6, 
                  hd.lsts[-1]*12/np.pi, hd.lsts[0]*12/np.pi))
plt.colorbar(label='Visibility Amplitude (Jy)')
plt.xlabel('Frequency (MHz)')
plt.ylabel('LST (hours)')
plt.title('Amplitude')

# Plot phase of visibility
plt.subplot(122)
plt.imshow(np.angle(data[bl]), aspect='auto', 
          extent=(hd.freqs[0]/1e6, hd.freqs[-1]/1e6, 
                  hd.lsts[-1]*12/np.pi, hd.lsts[0]*12/np.pi))
plt.colorbar(label='Visibility Phase')
plt.xlabel('Frequency (MHz)')
plt.ylabel('LST (hours)')
plt.title('Phase')

plt.tight_layout();

Some features to note from the waterfall plots:
* RFI
* Band edges
* Frequency and temporal structure

Thankfully, someone has already done a bunch of work to figure out which data might be bad and flagged it. Here is what the flags look like:

In [ ]:
bl = (65, 71, 'xx')

# Plot RFI Flags
plt.figure(figsize=(8,4))
plt.subplot(121)
plt.imshow(flags[bl].astype(float), aspect='auto',
          extent=(hd.freqs[0]/1e6, hd.freqs[-1]/1e6, 
                  hd.lsts[-1]*12/np.pi, hd.lsts[0]*12/np.pi))
plt.colorbar()
plt.xlabel('Frequency (MHz)')
plt.ylabel('LST (hours)')
plt.title('Flags')

# Plot phase of visibility
plt.subplot(122)
plt.imshow(np.abs(data[bl]) / ~flags[bl], aspect='auto',
          extent=(hd.freqs[0]/1e6, hd.freqs[-1]/1e6, 
                  hd.lsts[-1]*12/np.pi, hd.lsts[0]*12/np.pi))
plt.colorbar(label='Visibility Phase')
plt.xlabel('Frequency (MHz)')
plt.ylabel('LST (hours)')
plt.title('Flagged Phase')

plt.tight_layout();

Exercise: Now try a short baseline and a long baseline of the same orientation. Make waterfall plots (in both amplitude and phase) for both. How do they differ?

In [ ]:
bl1 = (37, 38, 'xx') # short E/W baseline
bl2 = (65, 71, 'xx') # long E/W baseline

# your answer here


Exercise: Now try comparing a N/S baseline to a E/W one of similar lengths do the waterfall plots differ for an E/W baseline vs. a N/S baseline? Does this make sense?

In [ ]:
bl1 = (37, 38, 'xx') # short E/W baseline
bl2 = (51, 83, 'xx') # short N/S baseline

# your answer here


# D) The Delay Transform

The delay transform is a clever technique we use to isolate bright foregrounds in our data (which we can then filter out). The delay transform is simply the Fourier transform of the visibility along frequency. 

Exercise: Try implementing the delay transform using `np.fft.fft` by following the steps below. We will think about its interpretation after.

In [ ]:
bl = (51,83,'xx')
vis = data[bl] * ~flags[bl]

#vis_dt = # your answer here 
vis_dt = np.fft.fftshift(np.fft.fft(vis,axis=1), axes=1) # Fourier-transform along frequency

# 2) Find the frequency width of a channel in GHz
#freq_width = # your answer here
freq_width =  np.median(np.diff(hd.freqs)) / 1e9 # GHz

#3) Convert frequencies to delays. Numpy's fftfreq function takes two arguments: 
#   the number of frequencies, and the frequeny width you calculated above
#delays = # your answer here
delays = np.fft.fftshift(np.fft.fftfreq(len(hd.freqs), freq_width)) # ns

import matplotlib
plt.figure()
plt.imshow(np.abs(vis_dt), aspect='auto', norm=matplotlib.colors.LogNorm(1e0,1e7),
           extent=(np.min(delays), np.max(delays),
                   hd.lsts[-1]*12/np.pi, hd.lsts[0]*12/np.pi))
plt.colorbar(label='Visibility Amplitude (Jy)')
plt.xlabel('Delay (ns)')
plt.ylabel('LST (hours)')
plt.title('Delay Transform');

Yuck! What happened? All that bright RFI - which are very well localized in frequency space - has spread out in delay space, causing that bright interference patten. So let's flag out the RFI!

Exericse: Now plot the delay transform again, but this time set the flagged channels to 0 by multiplying the visibility data by `~flags` (the tilde is to invert the flags, because the saved flags have 1's where the flags are).

In [ ]:
bl = (51,83,'xx')

# your answer here


That's better. So what do we see here? All the bright stuff at low delay values correspond to bright foreground sources that are smooth in frequency (and therefore "peaky" in delay). This is nice for us because we've isolated the foregrounds and we can filter them out easily in this space.

Let's think about what "delay" means physically. A delay can be thought of as the time difference between when a lightwave hits one antenna and when it hits the second antenna. In other words, there is a time lag between light hitting each antenna depending on the direction it comes from and the orientation of the baseline.

Where would the light have to be coming from for the time delay to be zero? Where would it need to be coming from to produce a maximum delay (hint: we call this the horizon limit)?

Exercise: For the baseline we picked earlier, calculate the theoretical maximum delay (in nanoseconds), using $t = d/c$, where $t$ is the time delay, $d$ is the baseline distance, and $c$ is the speed of light. You can calculate the baseline distance using the saved `antpos` and `ants` variables from earlier, or approximate it using the antenna layout plot.

In [ ]:
bl = (51,83,'xx')
# your answer here


A cool trick for a faster calculation is to convert your distance $d$ to feet. That number is also approximately the time delay in nanoseconds! 

Also note that the foregrounds in our delay-transform plot spread out past this horizon limit. This is due to all sorts of effects, but the biggest one is the RFI gaps. However, we have algorithms that can deal with these in a smarter way than just setting them to 0. 

In [ ]:
from hera_cal import delay_filter

bl = (51,83,'xx')
df = delay_filter.DelayFilter(hd) # establish delay filter object
df.run_filter(to_filter=[bl]) # filter the specific key we want (otherwise it takes a long time to do all keys)

vis_df = df.clean_resid[bl] # filtered visibility (i.e. the cleaned visibility)
clean = df.clean_data[bl] # low delay modes we don't want (i.e. the stuff that gets filtered out)

# Plot
plt.figure(figsize=(10,3))
plt.subplot(131)
plt.imshow(np.abs(data[bl] / ~flags[bl]), aspect='auto', norm=matplotlib.colors.LogNorm(1e0,1e2),
          extent=(hd.freqs[0]/1e6, hd.freqs[-1]/1e6, 
                  hd.lsts[-1]*12/np.pi, hd.lsts[0]*12/np.pi))
plt.colorbar(label='Visibility Amplitude (Jy)')
plt.xlabel('Frequency (MHz)')
plt.ylabel('LST (hours)')
plt.title('Original Visibility')

plt.subplot(132)
plt.imshow(np.abs(vis_df), aspect='auto', norm=matplotlib.colors.LogNorm(1e0,1e2),
          extent=(hd.freqs[0]/1e6, hd.freqs[-1]/1e6, 
                  hd.lsts[-1]*12/np.pi, hd.lsts[0]*12/np.pi))
plt.colorbar(label='Visibility Amplitude (Jy)')
plt.xlabel('Frequency (MHz)')
plt.ylabel('LST (hours)')
plt.title('Cleaned Visibility')

plt.subplot(133)
plt.imshow(np.abs(clean), aspect='auto', norm=matplotlib.colors.LogNorm(1e0,1e2),
          extent=(hd.freqs[0]/1e6, hd.freqs[-1]/1e6, 
                  hd.lsts[-1]*12/np.pi, hd.lsts[0]*12/np.pi))
plt.colorbar(label='Visibility Phase')
plt.xlabel('Frequency (MHz)')
plt.ylabel('LST (hours)')
plt.title('CLEAN Components')

plt.tight_layout();

Comparing the left and middle plots, we see that we've reduced the amplitude of our visibility by a couple orders of magnitude! We also see that some of the foreground structure we see in the left plot has been removed, as the middle plot looks more noise-like. Delay-filtering is a powerful tool for HERA; afterall, we are trying to find a tiny signal (the 21cm EoR signal) buried underneath a lot of stuff we don't care about!

Exercise. Try running the delay filter with `to_filter=[bl]` and `min_dly=6000` to use the CLEAN part of the filter without the "filtering" part. Now image the FFT of `df.clean_data[bl]` in delay vs. LST. How does this compare to what you did above?

In [ ]:
bl = (51,83,'xx')

# your answer here
